In [1]:
from ortools.linear_solver import pywraplp
import pandas as pd
"""NUI Galway CT5141 Assignment 1 : Unit Commitment

Student name(s): Galvin Venugopal, Atharva Kulkarni
Student ID(s):     (20235245)        (20231773)

We have a country with 10 generators of 4 types: Hydroelectric, Solid fuel, Gas, Solar
* Minimize cost of electricity production per hour
* How much electricity does each generator produce per hour

Decision variables

* A_1 to A_24 is the quantity of electricity produced by generator A during each hour of the day
* B_1 to B_24 is the quantity of electricity produced by generator B during each hour of the day
* C_1 to C_24 is the quantity of electricity produced by generator C during each hour of the day
* D_1 to D_24 is the quantity of electricity produced by generator D during each hour of the day
* E is the quantity of electricity produced by generator E during each hour of the day
* F is the quantity of electricity produced by generator F during each hour of the day
* G is the quantity of electricity produced by generator G during each hour of the day
* H_1 to H_24 is the quantity of electricity produced by generator H during each hour of the day
* I_1 to I_24 is the quantity of electricity produced by generator I during each hour of the day
* J_1 to J_24 is the quantity of electricity produced by generator J during each hour of the day

Objective Function

To minimize the cost of the electricity generated
(1.4*(A_1+...+A_24))+(1.4*(B_1+...+B_24))+(1.4*(C_1+...+C_24))+(1.4*(D_1+...+D_24))+(4.4*(E))+
(4.4*(F))+(4.4*(G))+(9.1*(H_1+...+H_24))+(6.6*(I_1+...+I_24))+(6.6*(J_1+...+J_24))

Constraints

10  <= A_1..A_24 <= 100
10  <= B_1..B_24 <= 80
10  <= C_1..C_24 <= 60
1   <= D_1..D_24 <= 10
100 <= E <= 900
100 <= F <= 600
10  <= G <= 100
100 <= H_1..H_24 <= 400
0   <= I_1..I_24 <= 70
0   <= J_1..J_24 <= 20

All Solar Generators (I,J) produce different levels based on the amount of sunlight available.
Total electricity generated in an hour by all the generators should be equal to the demand

"""
def GetInfo(filename):
    df = pd.read_csv(filename, header=None, usecols=[0])
    df.columns = ["Info"]
    information = df["Info"].values.tolist()
    return information

    
def Generator():
    
    # Instantiate an LP solver using the GLOP solver
    solver = pywraplp.Solver('Generator',
                             pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    LB = [10, 10, 10, 1, 100, 100, 10, 100, 0, 0]
    UB = [100, 80, 60, 10, 900, 600, 100, 400, 70, 20]
    Coeff = [1.4, 1.4, 1.4, 1.4, 4.4, 4.4, 4.4, 9.1, 6.6, 6.6]
    num_hours = 24
    Demand = GetInfo("demand.csv")
    Solarcurve = GetInfo("solar_curve.csv")
    
    #DVs for Generator A
    A = [solver.NumVar(LB[0], UB[0], f"A{i}") for i in range(num_hours)]
    B = [solver.NumVar(LB[1], UB[1], f"B{i}") for i in range(num_hours)]
    C = [solver.NumVar(LB[2], UB[2], f"C{i}") for i in range(num_hours)]
    D = [solver.NumVar(LB[3], UB[3], f"D{i}") for i in range(num_hours)]
    E = solver.NumVar(LB[4], UB[4], "E")
    F = solver.NumVar(LB[5], UB[5], "F")
    G = solver.NumVar(LB[6], UB[6], "G")
    H = [solver.NumVar(LB[7], UB[7], f"H{i}") for i in range(num_hours)]
    I = [solver.NumVar(LB[8], UB[8], f"I{i}") for i in range(num_hours)]
    J = [solver.NumVar(LB[9], UB[9], f"J{i}") for i in range(num_hours)]
    
    #Constraints
    #for j in range(num_hours):
    for ai, bi, ci, di, hi, ii, ji, sol_i, dem_i in zip(A,B,C,D,H,I,J,Solarcurve, Demand):
        #solver.Add(A[j] + B[j] + C[j] + D[j] + E + F + G + H[j] + Solarcurve[j]*I[j] + Solarcurve[j]*H[j] == Demand[j])
        solver.Add(ai + bi + ci + di + E + F + G + hi + sol_i*ii + sol_i*ji == dem_i)
    print("Num of Constraints: ", solver.NumConstraints())
    
    #Objective
    objective = solver.Objective()
    for j in range(num_hours):
        objective.SetCoefficient(A[j], Coeff[0])
        objective.SetCoefficient(B[j], Coeff[1])
        objective.SetCoefficient(C[j], Coeff[2])
        objective.SetCoefficient(D[j], Coeff[3])
        objective.SetCoefficient(H[j], Coeff[7])
        objective.SetCoefficient(I[j], Coeff[8])
        objective.SetCoefficient(J[j], Coeff[9])
    
    objective.SetCoefficient(E, Coeff[4])
    objective.SetCoefficient(F, Coeff[5])
    objective.SetCoefficient(G, Coeff[6])
    objective.SetMinimization()
    
    result = solver.Solve()
    if result == solver.OPTIMAL:
        for v in solver.variables():
            print(f"{v.name()} = {v.solution_value():.5}; coefficient {objective.GetCoefficient(v):.5}")
            
        print(f"Optimal objective value = {solver.Objective().Value()}")
        
        print_solution_and_sensitivity(solver,objective)

    else:
        print("Problem is not feasible")


def print_solution_and_sensitivity(solver, objective):
    """This function prints the solution and extracts all the sensitivity information
    that is easily available in OR-Tools. If the problem is IP, no
    sensitivity information is available, so it just prints what it can."""

    # is this a continuous problem? we can do more sensitivity
    # analysis if so.
    continuous_problem = all(v.Integer() == False for v in solver.variables())

    # the *reduced cost* for a variable is the change in objective
    # coefficient for the variable which would be required to move the
    # location of the optimum
    
        
    for c, a in zip(solver.constraints(), solver.ComputeConstraintActivities()):
        eps = 0.0000001
        # a constraint is *binding* if it is actually preventing the
        # optimum from improving -- the constraint line goes through
        # the optimum. another way to see it: the activity equals the
        # bound. another way: the dual is zero (the dual exists only
        # for continuous problems). we print a "*" for binding
        # constraints.
        if continuous_problem:
            binding = "* " if abs(c.DualValue()) > eps else "  "
        else:
            binding = "* " if (abs(c.ub() - a) < eps or abs(a - c.lb()) < eps) else "  "

        ctxt = " + ".join(f"{c.GetCoefficient(v):.5}*{v.name()}"
                          for v in solver.variables())
        
        # the *dual value* aka *shadow price* of a constraint is the
        # amount our profit could improve if the RHS of the constraint
        # would improve by 1 unit. for non-binding constraints, the
        # dual is 0. 

        if continuous_problem:

            if c.lb() == float("-inf"):
                # it is a <= constraint
                print(f"{binding} {c.name()}: {ctxt} = {a:.5} <= {c.ub():.5}; dual {c.DualValue():.5}; slack {c.ub() - a:.5}")
            elif c.ub() == float("inf"):
                # it is a >= constraint
                print(f"{binding} {c.name()}: {ctxt} = {a:.5} >= {c.lb():.5}; dual {c.DualValue():.5}; slack {a - c.lb():.5}")
            else:
                raise ValueError
                
        else:
            if c.lb() == float("-inf"):
                # it is a <= constraint
                print(f"{binding} {c.name()}: {ctxt} = {a:.5} <= {c.ub():.5}; slack {c.ub() - a:.5}")
            elif c.ub() == float("inf"):
                # it is a >= constraint
                print(f"{binding} {c.name()}: {ctxt} = {a:.5} >= {c.lb():.5}; slack {a - c.lb():.5}")
            else:
                raise ValueError


Generator()

Num of Constraints:  24
A0 = 51.0; coefficient 1.4
A1 = 36.0; coefficient 1.4
A2 = 36.0; coefficient 1.4
A3 = 28.0; coefficient 1.4
A4 = 15.0; coefficient 1.4
A5 = 10.0; coefficient 1.4
A6 = 57.0; coefficient 1.4
A7 = 93.0; coefficient 1.4
A8 = 97.0; coefficient 1.4
A9 = 94.0; coefficient 1.4
A10 = 94.0; coefficient 1.4
A11 = 92.0; coefficient 1.4
A12 = 88.0; coefficient 1.4
A13 = 86.0; coefficient 1.4
A14 = 82.0; coefficient 1.4
A15 = 82.0; coefficient 1.4
A16 = 82.0; coefficient 1.4
A17 = 99.0; coefficient 1.4
A18 = 100.0; coefficient 1.4
A19 = 100.0; coefficient 1.4
A20 = 100.0; coefficient 1.4
A21 = 86.0; coefficient 1.4
A22 = 69.0; coefficient 1.4
A23 = 53.0; coefficient 1.4
B0 = 10.0; coefficient 1.4
B1 = 10.0; coefficient 1.4
B2 = 10.0; coefficient 1.4
B3 = 10.0; coefficient 1.4
B4 = 10.0; coefficient 1.4
B5 = 10.0; coefficient 1.4
B6 = 10.0; coefficient 1.4
B7 = 10.0; coefficient 1.4
B8 = 10.0; coefficient 1.4
B9 = 10.0; coefficient 1.4
B10 = 10.0; coefficient 1.4
B11 = 10.0; c

ValueError: 

In [2]:
c = [15, 10]
n = len(c)
print(n)
for x in range(n):
    print(x)

2
0
1


In [3]:
d = pd.read_csv("demand.csv", header=None, usecols=[0])
d.columns = ["Demand"]
d.head()
dem = d["Demand"].values.tolist()
print (dem)

[1461, 1446, 1446, 1438, 1425, 1420, 1467, 1503, 1507, 1504, 1504, 1502, 1498, 1496, 1492, 1492, 1492, 1509, 1518, 1517, 1515, 1496, 1479, 1463]


In [16]:
solver = pywraplp.Solver('Generator',
                             pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
help(solver.NumVar)

Help on method NumVar in module ortools.linear_solver.pywraplp:

NumVar(lb: 'double', ub: 'double', name: 'std::string const &') -> 'operations_research::MPVariable *' method of ortools.linear_solver.pywraplp.Solver instance



In [25]:
UB = [100, 80, 60, 10, 900, 600, 100, 400, 70, 20]
C = [6]

for j in range(10):
    for ui,ci in zip(UB,C):
    print(ui*ci)

600
